# Transparent Tool Calling

In [51]:
from datetime import datetime
import os

from langchain.tools import tool  # Create tools with the decorator
from langchain_openai import ChatOpenAI  # Chat model
from langchain.agents import initialize_agent, AgentType  # Agent API
from langchain.memory import ConversationBufferMemory  # Conversational memory

In [60]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

@tool
def get_current_date(dummy_arg: str = "") -> str:
    """
    Devuelve la fecha y hora actuales en formato legible.
    Acepta un argumento dummy que es ignorado.
    """
    now = datetime.now()
    return now.strftime("%A, %d de %B de %Y, %H:%M:%S")

tools = [get_current_date]

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 3. Inicializar el modelo de lenguaje
# ══════════════════════════════════════════════════════════════════════════════
# Usamos gpt-3.5-turbo con temperatura moderada
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7) 

# ══════════════════════════════════════════════════════════════════════════════
# 4. Configurar memoria para la conversación
# ══════════════════════════════════════════════════════════════════════════════
# ConversationBufferMemory almacena mensajes previos para que el agente conserve contexto.  [oai_citation:3‡LangChain](https://python.langchain.com/docs/versions/v0_3/?utm_source=chatgpt.com) [oai_citation:4‡LangChain Blog](https://blog.langchain.dev/structured-tools/?utm_source=chatgpt.com)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ══════════════════════════════════════════════════════════════════════════════
# 5. Inicializar el agente con la herramienta y la memoria
# ══════════════════════════════════════════════════════════════════════════════
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=False, # Set to True to see the agent's thought process
    memory=memory,
    handle_parsing_errors=True # Helps in gracefully handling unexpected outputs from LLM
)

In [65]:
# ══════════════════════════════════════════════════════════════════════════════
# 6. Conversación de ejemplo
# ══════════════════════════════════════════════════════════════════════════════
# El agente decidirá, en tiempo de ejecución, si debe llamar a get_current_date.
# Query 1
user_input_1 = "Hola, Mi nombre es Kike"
print(f"\nUsuario: {user_input_1}")
response_1 = agent_chain.invoke({"input": user_input_1})
print(f"Agente: {response_1['output']}")

# Query 2 - Should use the tool
user_input_2 = "¿Qué día es hoy?"
print(f"\nUsuario: {user_input_2}")
response_2 = agent_chain.invoke({"input": user_input_2})
print(f"Agente: {response_2['output']}")

user_input_4 = "¿Cuál es la capital de Francia?"
print(f"\nUsuario: {user_input_4}")
response_4 = agent_chain.invoke({"input": user_input_4})
print(f"Agente: {response_4['output']}")

user_input_3 = "¿Recuerdas como me llamo?"
print(f"\nUsuario: {user_input_3}")
response_3 = agent_chain.invoke({"input": user_input_3})
print(f"Agente: {response_3['output']}")


Usuario: Hola, Mi nombre es Kike


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "¡Hola, Kike! ¿En qué puedo ayudarte hoy?"
}
```

> Finished chain.
Agente: ¡Hola, Kike! ¿En qué puedo ayudarte hoy?

Usuario: ¿Qué día es hoy?


> Entering new AgentExecutor chain...
```json
{
    "action": "get_current_date",
    "action_input": "dummy"
}
```
Observation: Sunday, 01 de June de 2025, 02:31:44
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The current date and time is Sunday, 01 de June de 2025, 02:31:44"
}
```

> Finished chain.
Agente: The current date and time is Sunday, 01 de June de 2025, 02:31:44

Usuario: ¿Cuál es la capital de Francia?


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "La capital de Francia es París."
}
```

> Finished chain.
Agente: La capital de Francia es París.

Usuario: ¿Recuerdas como me llamo?


> Entering new AgentExecutor chain...
``